In [1]:
import numpy as np
import pandas as pd
#import litstudy as lst
import nltk
#import Lbl2Vec as vec
import pandas as pd
import numpy as np#for text pre-processing
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score# bag of words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer#for word embedding
import gensim
from gensim.models import Word2Vec

In [2]:
master = pd.read_csv("captive_wild_animals.csv")
master = master[["Title", "Authors", "Abstract", "Published Year", "Accepted"]]
master.isna().sum()

Title               0
Authors            22
Abstract          107
Published Year      1
Accepted            0
dtype: int64

In [ ]:
master = master.dropna(ignore_index=True)
master.isna().sum()

In [ ]:
master

In [ ]:
### remove stopwords and punctuation.
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize as tokenize

words = tokenize(master["abstract"][1])
stop_words = set(stopwords.words("english"))
extra = {",", ".", "(", ")", ";", ":", "=", "[0-9]"}
filtered_list = []
for word in words:
    if word.casefold() not in stop_words and word.casefold() not in extra:
         filtered_list.append(word)

In [28]:
### stemming
from nltk.stem import SnowballStemmer

stemmer = SnowballStemmer("english")
stemmed = [stemmer.stem(word) for word in filtered_list]

In [33]:
abstracts = master["abstract"]

In [41]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize as tokenize
from nltk.stem import SnowballStemmer

nlp_dict = {}
all_words = []
row = int(0)
for abstract in abstracts:
    words = tokenize(abstract)
    stop_words = set(stopwords.words("english"))
    extra = {",", ".", "(", ")", ";", ":", "="}
    filtered_list = []
    for word in words:
        if word.casefold() not in stop_words and word.casefold() not in extra:
             filtered_list.append(word)
    
    stemmer = SnowballStemmer("english")
    stemmed = [stemmer.stem(word) for word in filtered_list]
    nlp_dict[row] = stemmed
    all_words.append(stemmed)
    row += 1

In [4]:
#convert to lowercase, strip and remove punctuations
def preprocess(text):
    text = text.lower() 
    text=text.strip()  
    text=re.compile('<.*?>').sub('', text) 
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text)  
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text=re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text) 
    return text
 
# STOPWORD REMOVAL
def stopword(string):
    a= [i for i in string.split() if i not in stopwords.words('english')]
    return ' '.join(a)#LEMMATIZATION
# Initialize the lemmatizer
wl = WordNetLemmatizer()
 
# This is a helper function to map NTLK position tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN# Tokenize the sentence

def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(word_tokenize(string)) # Get position tags
    a=[wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)] # Map the position tag and lemmatize the word/token
    return " ".join(a)

<>:7: SyntaxWarning: invalid escape sequence '\s'
<>:7: SyntaxWarning: invalid escape sequence '\s'
C:\Users\harrg\AppData\Local\Temp\ipykernel_38308\925071319.py:7: SyntaxWarning: invalid escape sequence '\s'
  text = re.sub('\s+', ' ', text)


In [5]:
def finalpreprocess(string):
    return lemmatizer(stopword(preprocess(string)))

master['abstract_clean'] = master['Abstract'].apply(lambda x: finalpreprocess(x))

In [6]:
### generate features
# trimmed abstract word count
list = []
for row in range(len(master)):
    list.append(len(master["abstract_clean"][row].split()))
master.insert(5, "abs_len", list)

In [7]:
# num stopwords
list = []
for row in range(len(master)):
    list.append(len(master["Abstract"][row].split()) - len(master["abstract_clean"][row].split()))
master.insert(6, "n_stopwords", list)

In [8]:
# title length
list = []
for row in range(len(master)):
    list.append(len(master["Title"][row].split()))
master.insert(7, "title_len", list)

In [9]:
# trimmed abastract word length average
list = []
for row in range(len(master)):
    list.append(sum(len(word) for word in master["abstract_clean"][row].split()) / len(master["abstract_clean"][row].split()))
master.insert(8, "abs_word_len", list)

In [10]:
# age/year published
master['age'] = 2024 - master['Published Year'].astype(int)

In [13]:
# number of authors
list = []
for row in range(len(master)):
    list.append(len(master["Authors"][row].split(";")))
master.insert(11, "n_authors", list)

In [14]:
master

,Title,Authors,Abstract,Published Year,Accepted,abs_len,n_stopwords,title_len,abs_word_len,abstract_clean,age,n_authors
0,"One Health, emerging infectious diseases and w...",Cunningham AA; Daszak P; Wood JLN,"Infectious diseases affect people, domestic an...",2017,0,148,85,11,6.905405,infectious disease affect people domestic anim...,7,3
1,COVID-19-Zoonosis or Emerging Infectious Disease?,Haider N; Rothman-Ostrow P; Osman AY; Arruda L...,The World Health Organization defines a zoonos...,2020,0,156,93,5,6.602564,world health organization defines zoonosis inf...,4,16
2,The role of wildlife in emerging and re-emergi...,Bengis RG; Leighton FA; Fischer JR; Artois M; ...,There are huge numbers of wild animals distrib...,2004,0,235,149,9,7.161702,huge number wild animal distribute throughout ...,20,6
3,Global hotspots and correlates of emerging zoo...,Allen T; Murray KA; Zambrana-Torrelio C; Morse...,Zoonoses originating from wildlife represent a...,2017,0,105,45,8,6.952381,zoonosis originate wildlife represent signific...,7,9
4,"Brucellosis in humans--etiology, diagnostics, ...",Galińska EM; Zagórski J,Brucellosis in humans is a zoonosis of greatly...,2013,0,165,69,6,6.406061,brucellosis human zoonosis greatly varied clin...,11,2
...,...,...,...,...,...,...,...,...,...,...,...,...
5529,Occurrence of ten protozoan enteric pathogens ...,"Menu, E.; Davoust, B.; Mediannikov, O.; Akiana...",Non-human primate populations act as potential...,2021,1,118,62,11,7.372881,non human primate population act potential res...,3,10
5530,Molecular and serological detection of Toxopla...,Hsu ChinWei; Wang PaoJung; Huang PeiYun; Lien ...,Toxoplasmosis caused by Toxoplasma gondii affe...,2022,1,123,91,14,6.211382,toxoplasmosis cause toxoplasma gondii affect c...,2,10
5531,Sequence analysis of novel Staphylococcus aure...,"Monecke, S.; Roberts, M. C.; Braun, S. D.; Die...",Staphylococcus aureus is a widespread and comm...,2022,1,175,91,12,6.240000,staphylococcus aureus widespread common opport...,2,18
5532,Intestinal parasites in captive wild animals a...,Qiu HongYu; Yang HuiNing; Sun XiaoJing; Jin Ch...,Objective: In order to investigate the prevale...,2022,1,107,52,13,7.280374,objective order investigate prevalence intesti...,2,7


In [ ]:
# keyword based features (abstract_clean)
keywords = ['species', 'conservation', 'wildlife', 'farm', 'fur', 'game', 'ecotourism', 'wild', 'animal', 'farming', 'animals', 'zoo', 'zoonosis', 'zoonotic', 'virus', 'vector']
for word in keywords:
    list=[]
    for row in range(len(master)):
        list.append(int(word in master["abstract_clean"][row]))
    master.insert(12, f"abs_{word}", list)

In [33]:
# keyword based features (title)
for word in keywords:
    list=[]
    for row in range(len(master)):
        list.append(int(word in master["Title"][row]))
    master.insert(12, f"ttl_{word}", list)

In [ ]:
final = master[['Accepted', 'abs_len',
       'n_stopwords', 'title_len', 'abs_word_len', 'age',
       'n_authors', 'ttl_vector', 'ttl_virus', 'ttl_zoonotic', 'ttl_zoonosis',
       'ttl_zoo', 'ttl_animals', 'ttl_farming', 'ttl_animal', 'ttl_wild',
       'ttl_ecotourism', 'ttl_game', 'ttl_fur', 'ttl_farm', 'ttl_wildlife',
       'ttl_conservation', 'ttl_species', 'abs_vector', 'abs_virus',
       'abs_zoonotic', 'abs_zoonosis', 'abs_zoo', 'abs_animals', 'abs_farming',
       'abs_animal', 'abs_wild', 'abs_ecotourism', 'abs_game', 'abs_fur',
       'abs_farm', 'abs_wildlife', 'abs_conservation', 'abs_species']]

In [37]:
X = master[['abs_len',
       'n_stopwords', 'title_len', 'abs_word_len', 'age',
       'n_authors', 'ttl_vector', 'ttl_virus', 'ttl_zoonotic', 'ttl_zoonosis',
       'ttl_zoo', 'ttl_animals', 'ttl_farming', 'ttl_animal', 'ttl_wild',
       'ttl_ecotourism', 'ttl_game', 'ttl_fur', 'ttl_farm', 'ttl_wildlife',
       'ttl_conservation', 'ttl_species', 'abs_vector', 'abs_virus',
       'abs_zoonotic', 'abs_zoonosis', 'abs_zoo', 'abs_animals', 'abs_farming',
       'abs_animal', 'abs_wild', 'abs_ecotourism', 'abs_game', 'abs_fur',
       'abs_farm', 'abs_wildlife', 'abs_conservation', 'abs_species']]

y = master['Accepted']

In [197]:
#SPLITTING THE TRAINING DATASET INTO TRAIN AND TEST
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1, shuffle=True)#Word2Vec

In [198]:
# Initializing the Logistic Regression model
model = LogisticRegression(solver = 'liblinear', C=0.01, penalty = 'l2', class_weight={0: 1., 1: 75.})

# Training the model
model.fit(X_train, y_train)

# Making predictions on the test set
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:,1]

print(classification_report(y_test,y_pred))
print('Confusion Matrix:',confusion_matrix(y_test, y_pred))
 
fpr, tpr, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)
print('AUC:', roc_auc)

# Calculating accuracy
accuracy = accuracy_score(y_test, y_pred)

              precision    recall  f1-score   support

           0       1.00      0.14      0.25       527
           1       0.06      1.00      0.11        27

    accuracy                           0.18       554
   macro avg       0.53      0.57      0.18       554
weighted avg       0.95      0.18      0.24       554

Confusion Matrix: [[ 74 453]
 [  0  27]]
AUC: 0.8165015109986646
